# 环境准备

在谷歌Colab上运行，数据从谷歌云盘上加载

In [1]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import json
import openai
import os
with open("/content/drive/MyDrive/config/openai.json", "r") as f:
  config_data = json.load(f)

api_key = config_data.get("api_key")
openai.api_key = api_key

Mounted at /content/drive


In [3]:
def load_data(source_path):
  import os
  import shutil
  if not os.path.exists("/content/data"):
    os.mkdir("/content/data")
  file_name = os.path.basename(source_path)
  dest_path = os.path.abspath(os.path.join("/content/data", file_name))
  if not os.path.exists(dest_path):
    shutil.copy(source_path, dest_path)
  return dest_path

# Homework1

使用openai.ChatCompletion.create方法代替chat_completion_request实现本notebook所有功能

## 1、定于工具函数

In [4]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_api，通过 OpenAI Python 库调用 Chat Completions API
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_api(messages, functions=None, function_call_name=None, model=GPT_MODEL):
    try:
      if functions:
        if function_call_name and function_call_name != "none":
          return openai.ChatCompletion.create(
              model = model,
              messages = messages,
              functions = functions,
              function_call = {"name":function_call_name}
            )
        elif function_call_name and function_call_name == "none":
          return openai.ChatCompletion.create(
              model = model,
              messages = messages,
              functions = functions,
              function_call = "none"
            )
        else:
          return openai.ChatCompletion.create(
                model = model,
                messages = messages,
                functions = functions
              )
      return openai.ChatCompletion.create(
              model = model,
              messages = messages
            )
    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [5]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [6]:
# 定义转为assistant_message对象为字典的函数

def format_assistant_message_dict(assistant_message):
  if assistant_message.get("function_call"):
    return {"role": assistant_message.role, "content": None, "function_call": assistant_message.function_call}
  return {"role": assistant_message.role, "content": assistant_message.content}

## 2、定义函数列表，分别是获取当前天气和获取未来N天的天气

In [7]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 函数的名称
        "description": "Get the current weather",  # 函数的描述
        "parameters": {  # 定义该函数需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]


## 3、使用GPT模型生成函数和对应参数

### 3.1、询问当前天气

In [8]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

# 使用定义的chat_completion_api函数发起一个请求，传入messages和functions作为参数
response = chat_completion_api(messages, functions=functions)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = response["choices"][0]["message"]

assistant_message_dict = format_assistant_message_dict(assistant_message)


# 将助手的回复消息添加到messages列表中
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure! Can you please provide me with your current location?



### 3.2、增加用户位置信息后，再次询问

In [9]:
# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I'm in Shenzhen, China."  # 用户的消息内容
})

response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]

assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure! Can you please provide me with your current location?

user: I'm in Shenzhen, China.

assistant[function_call]: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Shenzhen, China\",\n  \"format\": \"celsius\"\n}"
}



### 3.3、询问深圳未来若干天的天气情况

In [10]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "what is the weather going to be like in Shenzhen, China over the next x days"  # 用户询问今天的天气情况
})

response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shenzhen, China over the next x days

assistant[content]: Sure, I can help you with that. Please provide me with the number of days you would like to get the weather forecast for.



###3.4、确定询问未来5天的天气情况

In [11]:
# 向messages列表添加一条用户角色的消息，用户指定接下来的天数为5天
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shenzhen, China over the next x days

assistant[content]: Sure, I can help you with that. Please provide me with the number of days you would like to get the weather forecast for.

user: 5 days

assistant[function_call]: {
  "name": "get_n_day_weather_forecast",
  "arguments": "{\n  \"location\": \"Shenzhen, China\",\n  \"format\": \"celsius\",\n  \"num_days\": 5\n}"
}



###3.5、强制使用特定函数

In [12]:
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today in ShenZhen，Chain"  # 用户询问今天的天气情况
})

response = chat_completion_api(messages, functions=functions, function_call_name = "get_n_day_weather_forecast")

assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today in ShenZhen，Chain

assistant[function_call]: {
  "name": "get_n_day_weather_forecast",
  "arguments": "{\n  \"location\": \"ShenZhen, China\",\n  \"format\": \"celsius\",\n  \"num_days\": 1\n}"
}



###3.6、不强制使用特定函数

In [13]:
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today in ShenZhen，Chain"  # 用户询问今天的天气情况
})

response = chat_completion_api(messages, functions=functions)

assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today in ShenZhen，Chain

assistant[function_call]: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"ShenZhen, China\",\n  \"format\": \"celsius\"\n}"
}



### 3.7、强制不使用函数

In [14]:
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today in ShenZhen，Chain"  # 用户询问今天的天气情况
})

response = chat_completion_api(messages, functions=functions, function_call_name = "none")

assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today in ShenZhen，Chain

assistant[content]: Sure, let me check the weather for you.



## 4、执行GPT模型生成的函数

### 4.1、定义一个执行SQL查询的函数

In [15]:
import sqlite3
data_path = load_data("/content/drive/MyDrive/data/AI_LLM_training_camp/function_call/chinook.db")
conn = sqlite3.connect(data_path)
print("Opened database successfully")

Opened database successfully


### 4.2、定义3个与数据库查询有关函数

In [16]:
def get_table_names(conn):
    """返回一个包含所有表名的列表"""
    table_names = []  # 创建一个空的表名列表
    # 执行SQL查询，获取数据库中所有表的名字
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # 遍历查询结果，并将每个表名添加到列表中
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  # 返回表名列表


def get_column_names(conn, table_name):
    """返回一个给定表的所有列名的列表"""
    column_names = []  # 创建一个空的列名列表
    # 执行SQL查询，获取表的所有列的信息
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    # 遍历查询结果，并将每个列名添加到列表中
    for col in columns:
        column_names.append(col[1])
    return column_names  # 返回列名列表


def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

### 4.3、定义函数功能列表，让 GPT 模型帮我们构造一个完整的 SQL 查询

In [17]:
# 获取数据库信息，并存储为字典列表
database_schema_dict = get_database_info(conn)

# 将数据库信息转换为字符串格式，方便后续使用
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

# 定义一个功能列表，其中包含一个功能字典，该字典定义了一个名为"ask_database"的功能，用于回答用户关于音乐的问题
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

### 4.4、定义SQL查询函数和执行函数

In [18]:
# 定义函数ask_database,用于实际执行SQL查询并返回结果
def ask_database(conn, query):
    """使用 query 来查询 SQLite 数据库的函数。这里的query是大语言模型生成的"""
    try:
        results = str(conn.execute(query).fetchall())  # 执行查询，并将结果转换为字符串
    except Exception as e:  # 如果查询失败，捕获异常并返回错误信息
        results = f"query failed with error: {e}"
    return results  # 返回查询结果

# 根据消息中的功能调用信息来执行相应的功能并获取结果
def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database"
    if message["function_call"]["name"] == "ask_database":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        query = json.loads(message["function_call"]["arguments"])["query"]
        # 使用 ask_database 函数执行查询，并获取结果
        results = ask_database(conn, query)
    else:
        # 如果功能调用的名称不是 "ask_database"，则返回错误信息
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果


In [19]:
# 创建一个空的消息列表
messages = []

# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT artists.Name, COUNT(tracks.TrackId) AS TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.Name ORDER BY TrackCount DESC LIMIT 5;\"\n}"
}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In [20]:
# 向消息列表中添加一个用户的问题，内容是 "What is the name of the album with the most tracks?"
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})


response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]


response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT artists.Name, COUNT(tracks.TrackId) AS TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.Name ORDER BY TrackCount DESC LIMIT 5;\"\n}"
}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT albums.Title, COUNT(tracks.TrackId) AS TrackCount FROM albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.Title ORDER BY TrackCount DESC LIMIT 1;\"\n}"
}

function (ask_database): [('Greatest Hits', 57)]



# Homework2

**使用类似搜索引擎API，实现完整可执行的天气查询应用**

## 1、定义函数列表，分别是获取当前天气和获取未来N天的天气

In [21]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 函数的名称
        "description": "Get the current weather",  # 函数的描述
        "parameters": {  # 定义该函数需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]

## 2、实现获取天气函数

通过OpenWeatherMap的API接口实现，需要先注册OpenWeatherMap账户，并获取一个API Key

In [29]:
import requests

# 从谷歌云盘中获取OpenWeather的API_KEY
def get_open_weather_api_key():
  from google.colab import drive
  drive.mount('/content/drive')

  import json
  import os
  with open("/content/drive/MyDrive/config/open_weather.json", "r") as f:
    config_data = json.load(f)
  return config_data.get("api_key")

# 获取城市的维度和经度
def get_city_position(city_name):
  url = "http://api.openweathermap.org/geo/1.0/direct"
  api_key = get_open_weather_api_key()
  params = {"q":city_name, "appid":api_key}
  response = requests.get(url, params=params)
  if response.status_code == 200:
    data = response.json()[0]
    return data.get("lat"), data.get("lon")
  else:
    error_info = "请求失败, 状态码：{}".format(response.status_code)
    raise ValueError(error_info)

# 获取位置当前的温度
def get_current_temperature(lat, lon):
  url = "https://api.openweathermap.org/data/2.5/weather"
  api_key = get_open_weather_api_key()
  params = {"lat":lat, "lon":lon, "appid":api_key}
  response = requests.get(url, params=params)
  if response.status_code == 200:
    data = response.json().get("main").get("temp") - 273.15
    return round(data, 2)
  else:
    error_info = "请求失败, 状态码：{}".format(response.status_code)
    raise ValueError(error_info)

# 获取位置未来几天的问题，这个API需要充值，就暂不实现

# 封装获取城市当前的温度
def get_current_weather(city_name, temperature_format="celsius"):
  lat, lon = get_city_position(city_name)
  temperature = get_current_temperature(lat, lon)
  if temperature_format == "fahrenheit":
    return 9.0/5 * temperature + 32
  return temperature

print(get_current_weather("ShenZhen"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
29.55


## 3、执行天气查询


In [35]:
def execute_function_call(message):
  if message["function_call"]["name"] == "get_current_weather":
    arguments = json.loads(message["function_call"]["arguments"])
    city_name = arguments["location"]
    temperature_format = arguments["format"]
    temperature = get_current_weather(city_name, temperature_format)
    results = f"{city_name} tody temperature is {temperature} in {temperature_format}"
  else:
    results = f"Error: function {message['function_call']['name']} does not exist"
  return results

In [38]:
# 创建一个空的消息列表
messages = []

messages.append({
    "role": "system",
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})


messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today in ShenZhen"  # 用户询问今天的天气情况
})

response = chat_completion_api(messages, functions=functions)
assistant_message = response["choices"][0]["message"]
assistant_message_dict = format_assistant_message_dict(assistant_message)
messages.append(assistant_message_dict)

if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

pretty_print_conversation(messages)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today in ShenZhen

assistant[function_call]: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"ShenZhen\",\n  \"format\": \"celsius\"\n}"
}

function (get_current_weather): ShenZhen tody temperature is 29.85 in celsius

